In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.datasets as datasets

import theano
import theano.tensor as T
from theano import function,grad,pp,shared

In [2]:
batch_size = 100
ds = datasets.load_iris()
train_data_x = ds['data'][:,0:2]
train_data_y = ds['target']

train_data_x = np.append(train_data_x[train_data_y == 0],train_data_x[train_data_y == 1],axis=0)
train_data_y = np.array(np.append(train_data_y[train_data_y == 0],train_data_y[train_data_y == 1]),
                        dtype=theano.config.floatX)
train_set_x = shared(value=train_data_x,name='train_set_x',borrow=True)
train_set_y = shared(value=train_data_y,name='train_set_y',borrow=True)


In [3]:
class LogisticRegression(object):
    def __init__(self,n_in,n_out):
        self.x = T.matrix('x')
        self.y = T.dvector('y')
        self.W = shared(
            value=np.random.uniform(low=-0.1,high=0.1,size=(n_in,n_out)),
            name='W',
            borrow=True
        )
        self.b = shared(
            value=np.random.uniform(low=-0.1,high=0.1,size=(n_out,)),
            name='b',
            borrow=True
        )
        self.cost = T.mean( (self.y.reshape((batch_size,1)) - T.nnet.sigmoid(T.dot(self.x ,self.W) + self.b)) ** 2)
        self.output = None
    
learning_rate = 0.1
index = T.iscalar('index')
model = LogisticRegression(2,1)
g_w = grad(model.cost,model.W)
g_b = grad(model.cost,model.b)

train = function(inputs=[index],outputs=model.cost,
         updates=[
                 (model.W, model.W - learning_rate * g_w),
                 (model.b, model.b - learning_rate * g_b)
                 ],
         givens=[(model.x,train_set_x[index * batch_size :(index+1) * batch_size ]),
                 (model.y,train_set_y[index * batch_size :(index+1) * batch_size ])])



In [ ]:
prev_cost = train(0)
for i in range(100000):
    cost = train(0)
    if np.abs(prev_cost - cost) < 0.001:
        break
    
print train(0)

plt.plot(train_data_x[train_data_y==0][:,0],train_data_x[train_data_y==0][:,1],'+r')
plt.plot(train_data_x[train_data_y==1][:,0],train_data_x[train_data_y==1][:,1],'+b')
params = model.W.get_value()
bias = model.b.get_value()
x = np.arange(np.min(train_data_x[:,0]),np.max(train_data_x[:,0]),0.05)
y = (params[0]*x+bias)/-params[1]
plt.plot(x,y)